In [ ]:
import time
import asyncio
import json
from pymongo import MongoClient
from bson.json_util import dumps
from indy import anoncreds, did, ledger, pool, wallet, IndyError

async def run():
    await pool.set_protocol_version(2)

    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except IndyError as ex:
        if ex.error_code == 306:
            pass
    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)


    print("==============================")
    print("=== Credential Schemas Setup ==")
    print("------------------------------")
   
    client = MongoClient(host='supplier_db', port=27017, username='root', password='pass', authSource="admin")
    db = client["supplier_db"]

    supplier = db.supplier_tb.find_one({"name": "Supplier"})

    print("\"Supplier\" -> Create \"Supplier Attestation\" Schema")
    supplier_certificate = {
        'name': 'Supplier-Certificate',
        'version': '0.2',
        'attributes': ['first_name', 'last_name', 'salary', 'employee_status', 'experience']
    }

    (supplier['supplier_certificate_schema_id'], supplier['supplier_certificate_schema']) = \
        await anoncreds.issuer_create_schema(supplier['did'], supplier_certificate['name'], supplier_certificate['version'],
                                             json.dumps(supplier_certificate['attributes']))
    supplier_certificate_schema_id = supplier['supplier_certificate_schema_id']

    print("\"Supplier\" -> Send \"Supplier-Certificate\" Schema to Ledger")
    await send_schema(supplier['pool'], supplier['wallet'], supplier['did'], supplier['supplier_certificate_schema'])

    print("\"Supplier\" -> Get from Ledger \"Supplier-Certificate\" Schema")
    (supplier['supplier_certificate_schema_id'], supplier['supplier_certificate_schema']) = \
        await get_schema(supplier['pool'], supplier['did'], supplier_certificate_schema_id)

    print("\"Supplier\" -> Create and store in Wallet \"Supplier Job-Certificate\" Credential Definition")
    supplier_certificate_cred_def = {
        'tag': 'TAG1',
        'type': 'CL',
        'config': {"support_revocation": False}
    }

    (supplier['supplier_certificate_cred_def_id'], supplier['supplier_certificate_cred_def']) = \
        await anoncreds.issuer_create_and_store_credential_def(supplier['wallet'], supplier['did'],
                                                               supplier['supplier_certificate_schema'],
                                                               supplier_certificate_cred_def['tag'],
                                                               supplier_certificate_cred_def['type'],
                                                               json.dumps(supplier_certificate_cred_def['config']))

    print("\"Supplier\" -> Send \"Supplier Supplier-Certificate\" Credential Definition to Ledger")
    await send_cred_def(supplier['pool'], supplier['wallet'], supplier['did'], supplier['supplier_certificate_cred_def'])



async def send_schema(pool_handle, wallet_handle, _did, schema):
    schema_request = await ledger.build_schema_request(_did, schema)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, schema_request)

async def get_schema(pool_handle, _did, schema_id):
    get_schema_request = await ledger.build_get_schema_request(_did, schema_id)
    get_schema_response = await ledger.submit_request(pool_handle, get_schema_request)
    return await ledger.parse_get_schema_response(get_schema_response)

async def send_cred_def(pool_handle, wallet_handle, _did, cred_def_json):
    cred_def_request = await ledger.build_cred_def_request(_did, cred_def_json)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, cred_def_request)

